# Warsztaty Python w Data Science

## API jako źródło danych
## Web Scraping 3 z 3
---

### - JSON
### - REST - co to jest
### - New York Time API
### - Dane w języku naturalnym: analiza
---

## JSON

In [ ]:
import json

data = {'year': [2010, 2011, 2012, 2011, 2012, 2010, 2011, 2012],
        'team': ['Bears', 'Bears', 'Bears', 'Packers', 'Packers', 'Lions',
                 'Lions', 'Lions'],
        'wins': [11, 8, 10, 15, 11, 6, 10, 4],
        'losses': [5, 8, 6, 1, 5, 10, 6, 12]}


In [ ]:
s = json.dumps(data)
s

In [ ]:
json.loads(s)

In [ ]:
data == json.loads(s)

---
## REST

In [ ]:
API_KEY = "tu wpisz swoj klucz"

https://developer.nytimes.com/get-started

In [ ]:
url = 'https://api.nytimes.com/svc/books/v3/lists/names.json?api-key=' + API_KEY

In [ ]:
response = requests.get(url).json()
response


---
## New York Time API

https://developer.nytimes.com/get-started


In [ ]:
url = 'https://api.nytimes.com/svc/books/v3/lists/names.json?api-key=' + API_KEY

In [ ]:
response = requests.get(url).json()

In [ ]:
response.keys()

In [ ]:
response['results'][:3]

In [ ]:
[l['list_name'] for l in response['results']]

In [ ]:
url = 'https://api.nytimes.com/svc/books/v3/lists/current/hardcover-fiction.json?api-key=' + API_KEY

In [ ]:
response = requests.get(url).json()
response.keys()

In [ ]:
response['results'].keys()

In [ ]:
response['results']['books'][0].keys()

In [ ]:
[ book['title'] for book in response['results']['books'][:15]]

In [ ]:
url = 'https://api.nytimes.com/svc/books/v3/lists/2023-01-31/hardcover-fiction.json?api-key=' + API_KEY

In [ ]:
response = requests.get(url).json()
[ book['title'] for book in response['results']['books'][:15 ]]

In [ ]:
url = 'https://api.nytimes.com/svc/books/v3/reviews.json?author=Stephen+King&api-key=' + API_KEY

In [ ]:
response = requests.get(url).json()

In [ ]:
response.keys()

In [ ]:
response['results'][:3]

In [ ]:
[ (book['book_title'], book['summary']) for book in response['results'][:30] if len(book['summary']) > 0]

---


In [ ]:
import os
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
from dateutil.relativedelta import relativedelta

In [ ]:
base_url = 'https://api.nytimes.com/svc/archive/v1/'
url = base_url + '/' + '2023' + '/' + "3" + '.json?api-key=' + YOUR_API_KEY
response = requests.get(url).json()


In [ ]:
response.keys()

In [ ]:
response['response'].keys()

In [ ]:
len(response['response']['docs'])

In [ ]:
response['response']['docs'][0]

In [ ]:
from porter import PorterStemmer

In [ ]:
stemmer = PorterStemmer()

In [ ]:
stemmer.stem('programming')

In [ ]:
stemmer.stem('errors')

In [ ]:
def is_valid(article):
    has_headline = type(article['headline']) == dict and 'main' in article['headline'].keys()
    return has_headline


def parse(response):
    data = {'headline': [],  
        'date': [], 
        'doc_type': [],
        'material_type': [],
        'section': [],
        'keywords': []}
    
    articles = response['response']['docs'] 
    for article in articles: # For each article, make sure it falls within our date range
        date = dateutil.parser.parse(article['pub_date']).date()
        if is_valid(article):
            data['date'].append(date)
            data['headline'].append(article['headline']['main']) 
            if 'section' in article:
                data['section'].append(article['section_name'])
            else:
                data['section'].append(None)
            data['doc_type'].append(article['document_type'])
            if 'type_of_material' in article: 
                data['material_type'].append(article['type_of_material'])
            else:
                data['material_type'].append(None)
            keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            data['keywords'].append(keywords)
    return pd.DataFrame(data) 

In [ ]:
df = parse(response)

In [ ]:
df

---
## Dane w języku naturalnym: analiza

In [ ]:
headlines = [ doc['headline']['main'] for doc in response['response']['docs']]

In [ ]:
headlines[:15]

In [ ]:
all_headlines = ' '.join(headlines)

In [ ]:
## Tokenizer

In [ ]:
import re

tokenizer = re.compile(r'[\W]+')
tokenized = tokenizer.split(all_headlines)
str(tokenized[:15])

In [ ]:
tokenized = [ word.lower() for word in tokenized]
print(f"Mamy {len(tokenized)} wyrazów")

In [ ]:
counter = {}

for w in tokenized:
    counter[w] = counter.get(w,0)+1

print(f"Mamy {len(counter.keys())} RÓŻNYCH wyrazów")

In [ ]:
counted_words= [ (word,cnt) for word,cnt in counter.items() ]
counted_words[:4]

In [ ]:
from operator import itemgetter

counted_words.sort(key=itemgetter(1), reverse=True)
counted_words[:20]

In [ ]:
counted_words[-20:]

In [ ]:
counts = [ x[1] for x in counted_words ]
len(counts)

In [ ]:
sum(counts)

In [ ]:
sum(counts[:107])

In [ ]:
counted_words[97:117]

In [ ]:
count_df = pd.DataFrame(counts[:120]).reset_index()
columns = list(count_df.columns)
columns[1] = 'count'
count_df.columns = columns
count_df

In [ ]:
columns = list(count_df.columns)
columns[1] = 'count'
count_df.columns = columns
count_df

In [ ]:
plt.figure(figsize=(20,10))
plt.style.use("dark_background")

chart = sns.lineplot(x='index',
                     y='count',
                     linestyle='-', marker='o',
                     palette='bright',  markeredgecolor="black",   
                     data=count_df
                    )
chart.set_ylabel('Ilość wystąpień')
chart.set_xlabel('Rank wyrazu')
chart.ticklabel_format(useOffset=False, style='plain')
chart.set_title('Wyrazy w artykułach NYT')
plt.show();

In [ ]:
plt.figure(figsize=(20,10))
plt.style.use("dark_background")

chart = sns.lineplot(x='index',
                     y='count',
                     linestyle='-', marker='o',
                     palette='bright',  markeredgecolor="black",   
                     data=count_df
                    )
chart.set(yscale='log')
chart.set_ylabel('Ilość wystąpień (logarytmicznie)')
chart.set_xlabel('Rank wyrazu')
chart.set_title('Wyrazy w artykułach NYT')
plt.show();